# Regresión lineal utilizando Tensorflow

## Predicción de precios de casas utilizando un modelo de regresión lineal simple


Un modelo de regresión lineal busca encontrar la ecuación lineal que minimice el error cuadrado medio.

La ecuación lineal:

$h(x) = wx + b$

donde: 

* $x$ : Variable independiente
* $w$ : Pendiente
* $b$ : Intercepto

Función de costo:

$C(w,b) = \frac{1}{2} \sum_{i=1}^{n} (y_i − h(x_i))^2$

donde:

* $y_i$ : Valor real de cada dato en el dataset
* $h(x_{i})$ : Valor predecido por el modelo

Para poder optimizar este modelo es necesario encontrar los valores de $w$ y $b$ que minimizan la función de costo.
Esto se realiza a través de un aprendizaje por medio de *gradient descent*.


In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import datetime
%load_ext tensorboard

In [2]:
df = np.load('proyecto_training_data.npy')


df = pd.DataFrame(data = df, columns = ["SalePrice", "OverallQual", "1stFlrSF", "TotRmsAbvGrd", "YearBuilt", "LotFrontage"])

#Información básica de dataframe
df.describe()

,SalePrice,OverallQual,1stFlrSF,TotRmsAbvGrd,YearBuilt,LotFrontage
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1201.000000
mean,180921.195890,6.099315,1162.626712,6.517808,1971.267808,70.049958
std,79442.502883,1.382997,386.587738,1.625393,30.202904,24.284752
min,34900.000000,1.000000,334.000000,2.000000,1872.000000,21.000000
25%,129975.000000,5.000000,882.000000,5.000000,1954.000000,59.000000
50%,163000.000000,6.000000,1087.000000,6.000000,1973.000000,69.000000
75%,214000.000000,7.000000,1391.250000,7.000000,2000.000000,80.000000
max,755000.000000,10.000000,4692.000000,14.000000,2010.000000,313.000000


In [3]:
# Split de datos de entrenamiento y test

#Random seed para reproducibilidad de los resultados

np.random.seed(42)

#Shuffle de datos
df = df.sample(frac = 1)

#Training and test split

train = int(0.8 * len(df))

dftrain = df[:train]
dftest = df[:-train]

print('Longitud de set de entrenamiento: ',len(dftrain), ', Longitud de set de prueba: ', len(dftest))


Longitud de set de entrenamiento:  1168 , Longitud de set de prueba:  292


## Modelo de regresión lineal

### Variable a utilizar para modelo de regresión lineal

Según el análisis exploratorio de datos las variables que se utilizarán para crear los modelos de regresión lineal son:

* ```OverallQual``` con un $R = 0.79$

Se estará utilizando esta variable para la creación del modelo de regresión lineal.


### Entrenamiento de modelo de regresión lineal mediante ***gradient descent***

El modelo de regresión lineal se entrenará utilizando la siguiente función de costo. El algoritmo de gradient descent calcula la derivada parcial para cada uno de los parámetros de la función de costo. En cada iteración se realiza una modificación pequeña proporcional a esas derivadas parciales calculadas. A continuación se presenta este concepto en notación matemática.


Función de costo:

$C(w,b) = \frac{1}{2m} \sum_{i=1}^{m} (y_{i} − h(x_i))^2$

donde:

* $y_{i}$ : Valor real de cada dato en el dataset
* $wx_{i}+b$ : Valor predecido por el modelo

Derivadas parciales:

$\frac{\partial m }{\partial f} = \frac{1}{n} \sum_{i=1}^{n} (y_i − h(x_i))*m$

$\frac{\partial b }{\partial f} = \frac{1}{n} \sum_{i=1}^{n} (y_i − h(x_i))$

*Gradient descent*:

$w = w - \alpha \frac{1}{n} \sum_{i=1}^{n} (y_i - h(x_i))*m)$

$b = b - \alpha \frac{1}{n} \sum_{i=1}^{n} (y_i − h(x_i))$

$\alpha$ = tasa de aprendizaje

Los valores de $m$ y $b$ son actualizados iterativamente hasta minimizar el error, en otras palabras, hasta encontrar la línea de regresión que minimiza el error entre los datos predecidos y los datos reales.


### Implementación de modelo de regresión lineal con Tensorflow




In [4]:
#Implementacion de algoritmo por medio de programación orientada a objetos



class LinearModel:
    def __init__ (self):
        # vector de weights, con 2 variables.
        self.w = tf.get_variable("weights", dtype = tf.float32, shape = [1,2], initializer = tf.zeros_initializer())
        
    # Funcion para generar prediccion de modelo
    def __call__(self, x):
        with tf.name_scope("model"):
            return tf.matmul(self.w,x)
    # Funcion de actualizacion de parametros por medio de gradient descent
    def update(self, x, y, learningrate):
        with tf.name_scope("error"):
            error = self.error(x,y)
            # Escalar de resumen
            error_summary = tf.summary.scalar("ErrorSummary", error)
        gradient = tf.gradients(error, [self.w])
        updated_w = tf.assign(self.w, self.w -learningrate * gradient[0])
        return updated_w, error, error_summary
    
    # Funcion para retornar MSE
    def error(self, x, y):
        error = 1/2 * tf.reduce_mean(tf.math.square(y - self(x)))
        return error

In [6]:
# Definicion de variables de dataframe a utilizar en modelos

y = dftrain["SalePrice"]
x = dftrain["OverallQual"]
x = np.array([x, np.ones_like(x)], dtype = "float64")

# Funcion para entrenamiento

def training(lr, epochs, frecprint):
    # String para definicion de experimento
    string = './graphs/'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S") +"_lr="+str(lr)+ "_epochs="+str(epochs)

    g = tf.Graph()
    with g.as_default():
        # Inicialización de objeto de clase LinearModel
        modelo = LinearModel()
        # Definicion de placeholders para input de data a grafo
        tensor_x = tf.placeholder(tf.float32, [2,len(dftrain["OverallQual"])], "tensor_x")
        tensor_y = tf.placeholder(tf.float32, [len(dftrain["SalePrice"])], "tensor_y")
        # Utilizacion de método update para hacer el entrenamiento
        update_parameters = modelo.update(tensor_x, tensor_y, lr)
        
        # Writer para utilizacion de tensorboard
        writer = tf.summary.FileWriter(string, g)
        
        with tf.train.MonitoredSession() as session:
            feed_dict = {tensor_x:x, tensor_y:y}
            for i in range(epochs+1):
                
                # Entrenamiento
                training = session.run(update_parameters, feed_dict = feed_dict)
            
                if (i)%frecprint == 0:
                    # Cálculo de pesos para print por cada frecprint epocas
                    weights = session.run(modelo.w, feed_dict = feed_dict)
                    # Agregar datos a writer para poder visualizarlos en tensorboard
                    writer.add_summary(training[2], i)
                    print("Epoch: ", i, "Weights: ", weights, "Cost: ", training[1])
                    print("-------------------------------------------------------------------------")
                    
            writer.close()

#### Representación de gráfica de modelo

A continuación se muestra la representación gráfica del modelo definido como un grafo. Dentro de cada uno de los namespaces utilizados se encuentran las operaciones realiadas para el cálculo del error, gradientes y pesos, respectivamente.


<img src="graph.png">

### Experimentos para determinar tasa de aprendizaje y épocas de entrenamiento óptimas

Habiendo desarrollado la función para el cálculo de un modelo de regresión lineal, procedemos a realizar los experimentos para determinar cual es la tasa de aprendizaje y cantidad de épocas óptimas para el entrenamiento de este modelo de regresión lineal.

#### Hipótesis

Suponiendo que tenemos un modelo de regresión lineal, con un único mínimo global en su función de costo, la rapidez con el que el modelo alcanza el error mínimo debería de depender principalmente del learning rate. Esto a su vez también condiciona el número de épocas necesarias de entrenamiento antes de llegar a un mínimo.

(En este caso no se está considerando el caso o la posibilidad de overfitting, pero dado que tenemos mucho más datos que parámetros a optimizar, la probabilidad de overfitting es baja)

Consideremos entonces que un learning rate de entre 0.1 y 0.01 es el adecuado por realizar, y la cantidad de iteraciones puede depender de este learning rate inicial.


In [8]:
#Experimento 1

training(0.01, 20000,1000)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Weights:  [[11938.992   1810.8187]] Cost:  19683643000.0
-------------------------------------------------------------------------
Epoch:  1000 Weights:  [[ 36015.01 -35339.1 ]] Cost:  1320914200.0
-------------------------------------------------------------------------
Epoch:  2000 Weights:  [[ 39826.996 -59773.3  ]] Cost:  1258487400.0
-------------------------------------------------------------------------
Epoch:  3000 Weights:  [[ 42159.406 -74723.664]] Cost:  1235116500.0
-------------------------------------------------------------------------
Epoch:  4000 Weights:  [[ 43586.51 -83871.2 ]] Cost:  1226366600.0
-------------------------------------------------------------------------
Epoch:  5000 Weights:  [[ 44459.7  -89468.19]] Cost:  1223091100.0
----------

In [9]:
#Experimento 2

training(0.02, 8000,1000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Weights:  [[23877.984   3621.6375]] Cost:  19683643000.0
-------------------------------------------------------------------------
Epoch:  1000 Weights:  [[ 39831.383 -59801.414]] Cost:  1258469600.0
-------------------------------------------------------------------------
Epoch:  2000 Weights:  [[ 43588.69 -83885.15]] Cost:  1226362000.0
-------------------------------------------------------------------------
Epoch:  3000 Weights:  [[ 44995.008 -92899.42 ]] Cost:  1221863700.0
-------------------------------------------------------------------------
Epoch:  4000 Weights:  [[ 45521.363 -96273.27 ]] Cost:  1221233500.0
-------------------------------------------------------------------------
Epoch:  5000 Weights:  [[ 45718.375 -97536.08 ]] Cost:  1221145900.0
-------------------------------------------------------------------------
Epoch:  6000 Weights:  [[ 

In [10]:
#Experimento 3

training(0.1, 1000,100)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Weights:  [[119389.93   18108.188]] Cost:  19683643000.0
-------------------------------------------------------------------------
Epoch:  100 Weights:  [[nan nan]] Cost:  nan
-------------------------------------------------------------------------
Epoch:  200 Weights:  [[nan nan]] Cost:  nan
-------------------------------------------------------------------------
Epoch:  300 Weights:  [[nan nan]] Cost:  nan
-------------------------------------------------------------------------
Epoch:  400 Weights:  [[nan nan]] Cost:  nan
-------------------------------------------------------------------------
Epoch:  500 Weights:  [[nan nan]] Cost:  nan
-------------------------------------------------------------------------
Epoch:  600 Weights:  [[nan nan]] Cost:  nan
-------------------------------------------------------------------------
Epoch:  700 Weights:  [[n

In [11]:
#Experimento 4

training(0.1, 2000,100)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Weights:  [[119389.93   18108.188]] Cost:  19683643000.0
-------------------------------------------------------------------------
Epoch:  100 Weights:  [[nan nan]] Cost:  nan
-------------------------------------------------------------------------
Epoch:  200 Weights:  [[nan nan]] Cost:  nan
-------------------------------------------------------------------------
Epoch:  300 Weights:  [[nan nan]] Cost:  nan
-------------------------------------------------------------------------
Epoch:  400 Weights:  [[nan nan]] Cost:  nan
-------------------------------------------------------------------------
Epoch:  500 Weights:  [[nan nan]] Cost:  nan
-------------------------------------------------------------------------
Epoch:  600 Weights:  [[nan nan]] Cost:  nan
-------------------------------------------------------------------------
Epoch:  700 Weights:  [[n

In [14]:
#Experimento 5

training(0.01, 1000,100)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Weights:  [[11938.992   1810.8187]] Cost:  19683643000.0
-------------------------------------------------------------------------
Epoch:  100 Weights:  [[30554.354   -337.1659]] Cost:  1462722300.0
-------------------------------------------------------------------------
Epoch:  200 Weights:  [[31286.93  -5032.858]] Cost:  1440115000.0
-------------------------------------------------------------------------
Epoch:  300 Weights:  [[31984.387 -9503.447]] Cost:  1419622400.0
-------------------------------------------------------------------------
Epoch:  400 Weights:  [[ 32648.412 -13759.732]] Cost:  1401048700.0
-------------------------------------------------------------------------
Epoch:  500 Weights:  [[ 33280.6   -17811.986]] Cost:  1384212200.0
-------------------------------------------------------------------------
Epoch:  600 Weights:  [[ 33882.49

In [13]:
#Experimento 6

training(0.05, 10000,1000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Weights:  [[59694.965  9054.094]] Cost:  19683643000.0
-------------------------------------------------------------------------
Epoch:  1000 Weights:  [[942727.94   50631.484]] Cost:  16447350000000.0
-------------------------------------------------------------------------
Epoch:  2000 Weights:  [[27050622.   4115502.5]] Cost:  1.4864234e+16
-------------------------------------------------------------------------
Epoch:  3000 Weights:  [[8.1190874e+08 1.2656056e+08]] Cost:  1.3434531e+19
-------------------------------------------------------------------------
Epoch:  4000 Weights:  [[2.4407663e+10 3.8077363e+09]] Cost:  1.2142514e+22
-------------------------------------------------------------------------
Epoch:  5000 Weights:  [[7.3377972e+11 1.1447791e+11]] Cost:  1.0974616e+25
-------------------------------------------------------------------------


In [15]:
#Experimento 7

training(0.02, 5000,1000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Weights:  [[23877.984   3621.6375]] Cost:  19683643000.0
-------------------------------------------------------------------------
Epoch:  1000 Weights:  [[ 39831.383 -59801.414]] Cost:  1258469600.0
-------------------------------------------------------------------------
Epoch:  2000 Weights:  [[ 43588.69 -83885.15]] Cost:  1226362000.0
-------------------------------------------------------------------------
Epoch:  3000 Weights:  [[ 44995.008 -92899.42 ]] Cost:  1221863700.0
-------------------------------------------------------------------------
Epoch:  4000 Weights:  [[ 45521.363 -96273.27 ]] Cost:  1221233500.0
-------------------------------------------------------------------------
Epoch:  5000 Weights:  [[ 45718.375 -97536.08 ]] Cost:  1221145900.0
-------------------------------------------------------------------------


In [16]:
#Experimento 8

training(0.03, 4000,1000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Weights:  [[35816.977  5432.456]] Cost:  19683643000.0
-------------------------------------------------------------------------
Epoch:  1000 Weights:  [[ 42165.664 -74763.79 ]] Cost:  1235096400.0
-------------------------------------------------------------------------
Epoch:  2000 Weights:  [[ 44996.035 -92906.01 ]] Cost:  1221862900.0
-------------------------------------------------------------------------
Epoch:  3000 Weights:  [[ 45643.906 -97058.76 ]] Cost:  1221169500.0
-------------------------------------------------------------------------
Epoch:  4000 Weights:  [[ 45792.203 -98009.305]] Cost:  1221133200.0
-------------------------------------------------------------------------


In [17]:
#Experimento 9

training(0.03, 5000,1000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Weights:  [[35816.977  5432.456]] Cost:  19683643000.0
-------------------------------------------------------------------------
Epoch:  1000 Weights:  [[ 42165.664 -74763.79 ]] Cost:  1235096400.0
-------------------------------------------------------------------------
Epoch:  2000 Weights:  [[ 44996.035 -92906.01 ]] Cost:  1221862900.0
-------------------------------------------------------------------------
Epoch:  3000 Weights:  [[ 45643.906 -97058.76 ]] Cost:  1221169500.0
-------------------------------------------------------------------------
Epoch:  4000 Weights:  [[ 45792.203 -98009.305]] Cost:  1221133200.0
-------------------------------------------------------------------------
Epoch:  5000 Weights:  [[ 45826.15 -98226.91]] Cost:  1221131100.0
-------------------------------------------------------------------------


In [18]:
#Experimento 10

training(0.04, 3000,500)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Weights:  [[47755.97   7243.275]] Cost:  19683643000.0
-------------------------------------------------------------------------
Epoch:  500 Weights:  [[ 39840.19 -59857.88]] Cost:  1258433400.0
-------------------------------------------------------------------------
Epoch:  1000 Weights:  [[ 43593.082 -83913.3  ]] Cost:  1226351900.0
-------------------------------------------------------------------------
Epoch:  1500 Weights:  [[ 44997.05 -92912.51]] Cost:  1221861900.0
-------------------------------------------------------------------------
Epoch:  2000 Weights:  [[ 45522.29  -96279.195]] Cost:  1221233400.0
-------------------------------------------------------------------------
Epoch:  2500 Weights:  [[ 45718.773 -97538.64 ]] Cost:  1221145500.0
-------------------------------------------------------------------------
Epoch:  3000 Weights:  [[ 45792

In [19]:
#Experimento 11

training(0.04, 4000,500)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Weights:  [[47755.97   7243.275]] Cost:  19683643000.0
-------------------------------------------------------------------------
Epoch:  500 Weights:  [[ 39840.19 -59857.88]] Cost:  1258433400.0
-------------------------------------------------------------------------
Epoch:  1000 Weights:  [[ 43593.082 -83913.3  ]] Cost:  1226351900.0
-------------------------------------------------------------------------
Epoch:  1500 Weights:  [[ 44997.05 -92912.51]] Cost:  1221861900.0
-------------------------------------------------------------------------
Epoch:  2000 Weights:  [[ 45522.29  -96279.195]] Cost:  1221233400.0
-------------------------------------------------------------------------
Epoch:  2500 Weights:  [[ 45718.773 -97538.64 ]] Cost:  1221145500.0
-------------------------------------------------------------------------
Epoch:  3000 Weights:  [[ 45792

In [20]:
#Experimento 12

training(0.04, 4500,500)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Weights:  [[47755.97   7243.275]] Cost:  19683643000.0
-------------------------------------------------------------------------
Epoch:  500 Weights:  [[ 39840.19 -59857.88]] Cost:  1258433400.0
-------------------------------------------------------------------------
Epoch:  1000 Weights:  [[ 43593.082 -83913.3  ]] Cost:  1226351900.0
-------------------------------------------------------------------------
Epoch:  1500 Weights:  [[ 44997.05 -92912.51]] Cost:  1221861900.0
-------------------------------------------------------------------------
Epoch:  2000 Weights:  [[ 45522.29  -96279.195]] Cost:  1221233400.0
-------------------------------------------------------------------------
Epoch:  2500 Weights:  [[ 45718.773 -97538.64 ]] Cost:  1221145500.0
-------------------------------------------------------------------------
Epoch:  3000 Weights:  [[ 45792

In [7]:
%tensorboard --logdir ./graphs --port 6006

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 10992.

In [22]:
%tensorboard --logdir ./graphs --port 6006

Reusing TensorBoard on port 6006 (pid 9408), started 11:34:24 ago. (Use '!kill 9408' to kill it.)

### Resultados


Tras la realización de los experimentos se observaron los siguientes resultados:

* Un *learning rate* de 0.04 es cercano al máximo permitido antes de que el algoritmo de gradient descent empiece a diverger.
* Un *learning rate* mayor de 0.05 causa divergencia o resultados NaN.
* Tomando en cuenta que 0.04 es un *learning rate* que permite tomar pasos un poco más grandes, son necesarias menos iteraciones de la misma para llegar al mínimo de la función de costo.

### Conclusión

* Es posible hacer un entrenamiento por medio de *gradient descent* para un modelo de regresión lineal por medio de un paradigma de programación por grafos utilizando Tensorflow.
* El *learning rate* 0.04 con 4000-4500 épocas de entrenamiento fue lo necesario para optimizar el modelo en un costo mínimo, sin embargo, podría tomarse en cuenta el caso de entrenar por menos iteraciones, ya que cada subsecuente época de entrenamiento luego de las primeras 500-1000 causa una reducción pequeña del error.

<img src="relevantexperiments.png">